In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import copy
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.utils.data as data_utils
import math
import torchvision.models as models
from PIL import Image
import os

In [ ]:
def iter_gradients(x):
    if isinstance(x, Variable):
        if x.requires_grad:
            yield x.grad.data
    else:
        for elem in x:
            for result in iter_gradients(elem):
                yield result

In [ ]:
def zero_gradients(i):
    for t in iter_gradients(i):
        t.zero_()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d lijiyu/imagenet

100% 6.21G/6.21G [04:50<00:00, 25.1MB/s]
100% 6.21G/6.21G [04:50<00:00, 23.0MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imagenet.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
def mydeepfool(image, net, num_classes=10, overshoot=0.02, max_iter=50):

    """
       :param image: Image of size HxWx3
       :param net: network (input: images, output: values of activation **BEFORE** softmax).
       :param num_classes: num_classes (limits the number of classes to test against, by default = 10)
       :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
       :param max_iter: maximum number of iterations for deepfool (default = 50)
       :return: minimal perturbation that fools the classifier, number of iterations that it required, new estimated_label and perturbed image
    """
    is_cuda = torch.cuda.is_available()

    if is_cuda:
        # print("Using GPU")
        image = image.cuda()
        net = net.cuda()


    f_image = net.forward(Variable(image[None, :, :, :], requires_grad=True)).data.cpu().numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]

    I = I[0:num_classes]
    label = I[0]

    input_shape = image.cpu().numpy().shape
    pert_image = copy.deepcopy(image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    x = Variable(pert_image[None, :], requires_grad=True)
    fs = net.forward(x)
    fs_list = [fs[0,I[k]] for k in range(num_classes)]
    k_i = label

    # print(num_classes)

    pert = np.inf
    fs[0, I[0]].backward(retain_graph=True)

    grad_orig = x.grad.data.cpu().numpy().copy()

    for k in range(1, num_classes):
        zero_gradients(x)

        fs[0, I[k]].backward(retain_graph=True)
        cur_grad = x.grad.data.cpu().numpy().copy()

        # set new w_k and new f_k
        w_k = cur_grad - grad_orig
        f_k = (fs[0, I[k]] - fs[0, I[0]]).data.cpu().numpy()

        pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())

        # determine which w_k to use
        if pert_k < pert:
            pert = pert_k
            w = w_k

    # compute r_i and r_tot
    # Added 1e-4 for numerical stability
    loop_i = 0
    l = 1
    r = max_iter
    ans = 0
    while l<=r:
        mid = int((l+r)/2)
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(mid*r_i)
        if is_cuda:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()
        else:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot)

        y = Variable(pert_image, requires_grad=True)
        fs = net.forward(y)
        k_i = np.argmax(fs.data.cpu().numpy().flatten())
        loop_i += 1
        if k_i == label:
          l = mid + 1
        else:
          r = mid - 1
          ans = mid


    r_tot = np.float32(ans*r_i)
    r_tot = (1+overshoot)*r_tot

    if is_cuda:
        pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()
    else:
        pert_image = image + (1+overshoot)*torch.from_numpy(r_tot)
    y = Variable(pert_image, requires_grad=True)
    fs = net.forward(y)
    k_i = np.argmax(fs.data.cpu().numpy().flatten())

    return r_tot,loop_i, ans, label, k_i, pert_image

In [ ]:
def deepfool_batch(net, images, num_classes=1000, overshoot=0.02, max_iter=50, batch_size=64):
    """
       :param images: a batch of images of size NxHxWx3
       :param net: network (input: images, output: values of activation **BEFORE** softmax).
       :param num_classes: num_classes (limits the number of classes to test against, by default = 10)
       :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
       :param max_iter: maximum number of iterations for deepfool (default = 50)
       :return: minimal perturbation for each image, number of iterations that it required, original labels and new labels, and perturbed images
    """
    results = []
    count = 0
    total = 0

    for i in range(0, len(images), batch_size):
        print(i)
        batch = images[i:i + batch_size]

        is_cuda = torch.cuda.is_available()
        if is_cuda:
            # print("Using GPU")
            batch = batch.cuda()
            net = net.cuda()
        # else:
        #     print("Using CPU")

        batch_results = []
        for image in batch:
            # Apply deepfool to each image in the batch
            r_tot, iter, orig_label, new_label, pert_image = deepfool(image, net, num_classes, overshoot, max_iter)
            total += 1
            batch_results.append((r_tot, iter, orig_label, new_label, pert_image))
            if(orig_label != new_label):
              count += 1
        results.extend(batch_results)
    return results,count,total


In [ ]:
import torch
from torchvision import datasets, transforms
mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean,
                         std = std)])
dataset = datasets.ImageFolder('/content/imagenet', transform = transform)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=320, shuffle=True)

In [ ]:
# Looping through it, get a batch on each loop
for images, labels in dataloader:
    pass

In [ ]:
images, labels = next(iter(dataloader))
# images = images[0:320]
images.size()

torch.Size([320, 3, 224, 224])

In [ ]:
net = models.resnet34(pretrained=True)
net.eval()
results, count, total = deepfool_batch(net,images)  # 49 min
print("results = ",count) #310
print("total = ", total)  #320

0
64
128
192
256
results =  310
total =  320


In [ ]:
total_iter = 0

for i in range(320):
    total_iter += results[i][1]

print("total iteration = ", total_iter) # 1868

total iteration =  1868


In [ ]:
results, count, total = deepfool_batch(net,images)  # 2hrs 10 min

0
64
128
192
256


In [ ]:
print("results = ",count)
print("total = ", total)

results =  320
total =  320


In [ ]:
total_iter = 0

for i in range(320):
    total_iter += results[i][1]

print("total iteration = ", total_iter) # 836

total iteration =  836
